In [ ]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwc.multires.class_multiressolver import *
import matplotlib.pyplot as plt
import torch
from src.cpwc.tools.ptychography import Ptychography as Ptychography
from src.cpwc.tools.utils import *
torch.cuda.empty_cache()
from src.utils.cuda_pyth import empty_cuda, print_cuda_tensors
from src.utils.get_image import get_image
from src.utils.noise import add_gaussian_noise, add_poisson_noise
from src.utils.metrics import get_ring_average
from src.utils.manage_data import save_data,unwrap_2d,extract_data
# Set seeds 
torch.manual_seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
max_scale = 9
image,image_tensor_ = get_image("samples/potential.npy", max_scale,device)
max_probe_size = 128
cycle = [0, -1, -1, -1,  1, 1, 1]
tol = [1e-10] * 9
tol_in = [1e-15] * 9


In [ ]:
# % 75 Overlap w/ MRGD
LR = 1e-2
max_shift = 32
I_in = 25*np.array([1, 15, 10, 5, 10, 30, 100])
I_out = 10*np.array([0, 0, 0, 30, 10,10,300])
lmbda = 1e-9
noise = 1e6
linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)
multires = MultiRes(max_scale, device)
image_tensor__ = linOperator.apply(image_tensor_)
image_tensor__ = add_poisson_noise(image_tensor__, noise, 'cuda')
loss = Loss(linOperator,image_tensor__,lmbda= lmbda)
model = MultiResSolver(multires, 
                        loss, 
                        LR = LR,
                        I_in = I_in,
                        I_out = I_out,
                        tol = tol,
                        tol_in = tol_in,
                        cycle = cycle,
                        l1_type = "l1_row",
                        gt = image_tensor_)
model.solve_multigrid()
model.print_time()

if max_shift == 32:
    overlap = 75
elif max_shift == 64:
    overlap = 50
else:
    overlap = 25


In [ ]:
path = save_data(model = model,
          image_path= "samples/potential.npy",
          metrics = ['csim','loss','frc',"image"],
          device = "cuda",
          max_scale = 9,
          overlap = overlap,
          spline_type="cpwc",
          lambda_ = lmbda,
          noise_type="poisson",
          noise = noise,
          loop = "mrgd")

In [ ]:
image = np.load(path + "_image.npy")

In [ ]:
plt.imshow(image)
plt.axis("off")

In [ ]:

frc = np.load(path + "_frc.npy")

In [ ]:
plt.figure(dpi = 200)
plt.plot(frc)
plt.grid()

In [ ]:
frc1 = np.load("data/cpwc_overlap25_mrgd_lambda1e-09_noise_typepoisson_noise1000000.0_frc.npy")
frc2 = np.load("data/cpwc_overlap50_mrgd_lambda1e-09_noise_typepoisson_noise1000000.0_frc.npy")
frc3 = np.load("data/cpwc_overlap75_mrgd_lambda1e-09_noise_typepoisson_noise1000000.0_frc.npy")
plt.figure(dpi = 200)
plt.semilogy(frc1)
plt.semilogy(frc2)
plt.semilogy(frc3)
plt.grid()
plt.legend(["Overlap 25","Overlap 50","Overlap 75"])
plt.title("FRC")
plt.xlabel("Resolution")
plt.ylabel("Correlation")
